In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

# 定義輸入與輸出檔案
input_file = "/content/drive/MyDrive/Colab Notebooks/專題_錢幣辨識/期末的文字檔/input.txt"  # 輸入檔案名稱
image_folder = "/content/drive/MyDrive/Colab Notebooks/專題_錢幣辨識/期末"
output_file = "/content/drive/MyDrive/Colab Notebooks/專題_錢幣辨識/out.txt"  # 輸出檔案名稱

# 初始化影像路徑列表
image_paths = []

# 讀取 in.txt，解析影像路徑，檢查檔案是否存在
with open(input_file, 'r') as file:
    for line in file:
        parts = line.strip()
        image_path = os.path.join(image_folder, parts)  # 將路徑與檔名結合
        if os.path.exists(image_path):
            image_paths.append(image_path)
        else:
            print(f"找不到對應檔案，準備忽略 {image_path}，已跳過。")

# 確認有效影像數量
num_images = len(image_paths)
print(f"有效影像數量: {num_images}")

# 初始化儲存結果
all_results = []
output_images = []

# 處理每張影像
for image_path in image_paths:
    in_image = cv2.imread(image_path)
    if in_image is None:
        print(f"錯誤：無法載入影像 {image_path}")
        continue

    # 轉換為灰階並應用高斯模糊 (降噪處理)
    gray_image = cv2.cvtColor(in_image, cv2.COLOR_BGR2GRAY)


    # 應用自適應直方圖均衡化 (CLAHE) 增強對比度
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_gray = clahe.apply(gray_image)

    # 調整亮度和對比度
    alpha = 1.5  # 對比度調整係數
    beta = 20    # 亮度調整值
    adjusted_gray = cv2.convertScaleAbs(enhanced_gray, alpha=alpha, beta=beta)


    # 應用高斯模糊 (降噪處理)
    blurred = cv2.GaussianBlur(gray_image, (9, 9), 2)

    # 使用霍夫圓檢測
    circles = cv2.HoughCircles(
        blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=35,
        param1=100, param2=30, minRadius=20, maxRadius=70
    )

    # 儲存結果的列表
    output = in_image.copy()
    coin_areas = []

    if circles is not None:
        circles = np.uint16(np.around(circles))
        radii = []
        for idx, i in enumerate(circles[0, :]):
            x, y, radius = i
            radii.append(radius)
            coin_area_data = {"x": x, "y": y, "radius": radius}
            coin_areas.append(coin_area_data)

        # 根據半徑排序並分四等分
        radii.sort()
        n = len(radii) // 4
        thresholds = [radii[n - 1], radii[2 * n - 1], radii[3 * n - 1]]

        # 計算最大四等分的平均值
        max_avg = sum(radii[3 * n:]) / len(radii[3 * n:])

        # 過濾掉超過 20% 的圓
        filtered_coin_areas = []
        for coin in coin_areas:
            if coin["radius"] <= max_avg * 1.2:
                filtered_coin_areas.append(coin)

        # 更新硬幣類型
        counts = [0, 0, 0, 0]  # 初始化硬幣計數 [1元, 5元, 10元, 50元]
        for idx, coin in enumerate(filtered_coin_areas):
            radius = coin["radius"]
            if radius <= thresholds[0]:
                coin["type"] = "1"
                counts[0] += 1
            elif radius <= thresholds[1]:
                coin["type"] = "5"
                counts[1] += 1
            elif radius <= thresholds[2]:
                coin["type"] = "10"
                counts[2] += 1
            else:
                coin["type"] = "50"
                counts[3] += 1

            # 在輸出影像上繪製並標記硬幣
            x, y = coin["x"], coin["y"]
            cv2.circle(output, (x, y), radius, (0, 255, 0), 2)  # 繪製綠色圓圈
            cv2.putText(output, f"${coin['type']}", (x - 40, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2)  # 標記硬幣編號與類型

        # 儲存結果
        all_results.append(counts)
        output_images.append(output)

# 將結果寫入 out.txt
with open(output_file, "w") as file:
    file.write(f"{num_images}\n")
    for result in all_results:
        file.write(" ".join(map(str, result)) + "\n")

# 列印結果供檢查
print("檢測結果預覽：")
for idx, result in enumerate(all_results):
    print(f"影像 {idx + 1}: 1元={result[0]}個, 5元={result[1]}個, 10元={result[2]}個, 50元={result[3]}個")

# 單獨展示輸入影像和辨識後的影像
for idx, (image_path, output_image) in enumerate(zip(image_paths, output_images)):
    # 顯示輸入影像
    input_image = cv2.imread(image_path)
    if input_image is not None:
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(input_image)
        plt.axis('off')
        plt.title(f"Input {idx + 1}")

    # 顯示辨識後影像
    output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 2)
    plt.imshow(output_image)
    plt.axis('off')
    plt.title(f"Output {idx + 1}")

    plt.show()

# 檢查輸出檔案是否存在
if os.path.exists(output_file):
    print(f"輸出檔案 {output_file} 已保存成功！")
else:
    print(f"輸出檔案 {output_file} 保存失敗")


Output hidden; open in https://colab.research.google.com to view.